In [ ]:
import folium
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import os
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

mexico = gpd.read_file("D:/tierra/data/natural_earth/ne_110m_admin_0_countries.shp")
mexico = mexico[mexico['ADMIN'] == 'Mexico']

# Define grid size (approx. 10km in degrees for WGS84)
grid_size = 0.1  # ~10km

# Get bounding box of Mexico
minx, miny, maxx, maxy = mexico.total_bounds  # (west, south, east, north)

# Generate grid cells
grid_cells = []
x_coords = np.arange(minx, maxx, grid_size)
y_coords = np.arange(miny, maxy, grid_size)

for x in x_coords:
    for y in y_coords:
        grid_cells.append(Polygon([(x, y), (x + grid_size, y), (x + grid_size, y + grid_size), (x, y + grid_size)]))

# Convert grid to GeoDataFrame
grid_gdf = gpd.GeoDataFrame(geometry=grid_cells, crs="EPSG:4326")

# Clip the grid to match Mexico's boundaries
grid_gdf = gpd.overlay(grid_gdf, mexico, how="intersection")

# save the grid to a file
file_path = "D:/tierra/data/grids/mexico_grid_10km.shp"
os.makedirs(os.path.dirname(file_path), exist_ok=True)
grid_gdf.to_file(file_path, driver="ESRI Shapefile")

# Create Folium map
m = folium.Map(location=[mexico.geometry.centroid.y.values[0], mexico.geometry.centroid.x.values[0]], zoom_start=5, tiles="OpenStreetMap")

# Add clipped grid to the map
folium.GeoJson(
        grid_gdf, 
        name="10km Grid", 
        style_function=lambda x: {'color': 'blue', 'weight': 0.2, 'fillOpacity': 0},
        highlight_function=lambda x: {'weight': 3, 'fillOpacity': 0.5},
).add_to(m)

# Show map
m
